# OLS Regression Model

## 1. Practical verification of Ordinary Least Squares properties using simulations in Julia:
#### Loading installed packages


In [ ]:
using Random 
using DataFrames 
using Plots 
using Statistics 
using GLM # package for estimating Generalized Linear Models 
using Markdown

In [ ]:
Random.seed!(1); # Setting a seed for random number generating to ensure a replicability

### a. Synthetic data simulation according to linear regression (LR) Data Generating Process,

#### Defining Data Generating Process (DGP)

A regional convenience store sells among others a bottled still water. Historically, the store set a price for this bottled water in relation to outside temperature, i.e. the higher temperarutre the higher price.
You'll be provided with a historical demand data composed of: temperature, rainfall, price and sales volume.
Our task for today is to find out the relationship between price and sales volume, so the store will be able to set the optimal price based on this relationship.

*dgp()* function simulates weekly store demand data for bottled still water and is composed of following variables / columns:
- temperature - average of temperature in Celcius degrees, an exogenous variable, non-depndent on remaining variables:

    $temperature_t = 10 + 5 \times \epsilon_t^{temperature}$
- rainfall - a weekly average of rainfall, correlated positively with temperature:

    $rainfall_t = max(0, temperature_t - 10 + 5 \times \epsilon_t^{rainfall}$)
- price - in Dollars for a large pack, driven positively by an average temperarture:

    $price_t = 14 + \frac{1}{10} \times temperature_t + \epsilon_t^{price}$
- demand - sales volume in unites, dependent negatively on  price and positivaly on temperature:

    $demand_t = \beta_0 + \beta_1\times price_t+\beta_2\times temperature_t+\epsilon_t^{demand}$.

Where:
- $\epsilon_t^{variable} \overset{\text{iid}}{\sim} \mathcal{N}(0, 1)$, $variable \in \{temperature, rainfall, price, demand\}$,
- $t\in\{1, ..., N\}$, $N$ - number of historical weeks.

In [ ]:
function dgp(;
             N = 100, # number of observations
             β₀ = 30.0, # intercept / bias of linear regression, to be alligned with the rest of βs
             β₁ = -1.0, # 1st coefficient measuring the impact of price on demand / sales volume
             β₂ = 0.25) # 2nd coefficient measuring the impact of temperature on demand / sales volume 
    temperature = 10 .+ 5 * randn(N) 
    rainfall = max.(0, temperature .- 10 + 5 * randn(N)) # rainfall depedends positively on temperature
    price = 14 .+ temperature / 10 + randn(N) / 2 # price depends positively on temperature
    demand = β₀ .+ β₁ * price + β₂ * temperature  + randn(N) 
    return DataFrame(; price, temperature, rainfall, demand)
end 
# footnote: an alternative function is make_regression() from MLJ pakcage

Generating a single dataset:

In [ ]:
dataset = dgp()

### b. Visualization of simulated data

Histogram of independent temperature variable:

In [ ]:
histogram(dataset.temperature,
          xlab = "Temperature",
          bins = 15,
          legend = false)

Scatterplot of temperature (x) impacting rainfall and price (y's):

In [ ]:
scatter(dataset.temperature,
        Matrix(dataset[!, [:price,:rainfall]]),
        layout = (2, 1),
        xlabel = "Temperature",
        ylabel = ["Price" "Rainfall"],
        legend = false)

Pearson correlations between temperature and [rainfall, price]:


In [ ]:
cor(dataset.temperature,
    Matrix(dataset[!, [:price, :rainfall]]))

Scatterplot of demand vs temperature, rainfall and price:

In [ ]:
scatter(Matrix(dataset[!, 1:3]),
        dataset.demand,
        layout = (1, 3),
        ylabel = "Demand",
        xlabel = ["Price" "Temperature" "Rainfall"],
        legend = false)

Pearson correlations between demand and [temperature, rainfall, price]

In [ ]:
cor(Matrix(dataset[!, 1:3]),
     dataset.demand)

### c. Linear Regression estimation using Ordinary Least Squares (OLS)

#### Our own algebraic matrix implementation

Defining an estimating function:


In [ ]:
least_squares(x, y) =  [ones(size(y)) x] \ y

Using this function to estimate our linear regression:

In [ ]:
βs = least_squares(Matrix(dataset[!, 1:3]), dataset.demand)

#### GLM package implementation

Defining model specification


In [ ]:
specification = @formula(demand ~ price + temperature + rainfall)

Estimating a specified model on a given dataset:

In [ ]:
fitted_OLS = lm(specification, dataset)

R² calculation:


In [ ]:
R² = r2(fitted_OLS)

### e. Demonstration of unbiasedness OLS property using Monte Carlo (MC) simulations


Initializing empty arrays to be populated by MC simulations' results 

In [ ]:
estimation_columns =  ["β₀", "β₁", "β₂", "β₃", "β₃ p-value", "R²"]
estimation_results = DataFrame([column_name => Float64[] for column_name in estimation_columns])

Setting up a number of Monte Carlo simulations:

In [ ]:
N = 100_000

Running Monte Carlo Simulations:


In [ ]:
for i in 1:N
   dataset = dgp() # Data generation 
   fitted_OLS = lm(specification, dataset) # OLS fitting
   push!(estimation_results,
          [coef(fitted_OLS)[1:4] 
          DataFrame(coeftable(fitted_OLS))[4, 5]
          r2(fitted_OLS)]) 
end

In [ ]:
estimation_results

Sampling distribution of intercept OLS estimates

In [ ]:
histogram(estimation_results.β₀,
    title = "Monte Carlo approximation of
    sampling distribution of intercept OLS estimates",
    xlabel = "Intercept",
    legend = false)

Key takeaways from sampling distribution of OLS intercept (β₀) estimates:

In [ ]:
display(md"""
The intercept (β₀) estimated by Ordianary Least Squares (OLS) at the value of 
**$(round(DataFrame(coeftable(fitted_OLS))[1, 2], sigdigits = 3))**,
can be considered as a random variable derived from so called sampling distribution that 
we approximated (see histogram) through Monte Carlo simulations with:
1. a mean of **$(round(mean(estimation_results.β₀), sigdigits = 3))**
    equal to the true value of this parameter (unbiasedness property of an estimator),
2. a standard deviation (called standard error of this estimate) of **$(round(std(estimation_results.β₀), sigdigits = 3))** 
    being estimated by OLS to be 
    **$(round(DataFrame(coeftable(fitted_OLS))[1, 3],sigdigits = 3))**
    which is the minimal possible standard error (efficiency property of an estimator)
3. 95%-Confidence Interval of
    **$(string(round.(quantile(estimation_results.β₀, [0.025, 0.975]), sigdigits = 3)))**
    being estimated by OLS to be **$(string(round.(Vector(DataFrame(coeftable(fitted_OLS))[1, 6:7]), sigdigits = 3)))**.

Let us have a look at a last complete table:
<div>
$(PrettyTables.pretty_table(String, DataFrame(coeftable(fitted_OLS)),tf=PrettyTables.tf_html_dark))
</div>
""")


Sampling distribution of temperature, price and rainfall OLS estimates:

In [ ]:
histogram(Matrix(estimation_results[!,2:4]),
    bins = 25, xlabel = "Estimate",
    labels = ["β₁ (price)" "β₂ (temperature)" "β₃ (rainfall)"],
    title = "MC approxim. of sampling distributions of
    temperature, price and rainfall OLS estimates")

Key takeaways from sampling distribution of OLS temperature estimates

In [ ]:
display(md"""
The price impact coefficient (β₁) estimated by Ordianary Least Squares (OLS) at the value of ≈ **$(round(DataFrame(coeftable(fitted_OLS))[2, 2], sigdigits = 3))** can be considered as a random variable derived from so called sampling distribution that we approximated (see a blue histogram) through Monte Carlo simulations with:
1. a mean of ≈ **$(round(mean(estimation_results.β₁), sigdigits = 3))** equal to the true value of this parameter (unbiasedness property of an estimator),
2. a standard deviation (called standard error of this estimate) of ≈**$(round(std(estimation_results.β₁), sigdigits = 3))** being estimated by OLS to be **$(round(DataFrame(coeftable(fitted_OLS))[2, 3], sigdigits = 3))** - the minimal possible standard error (efficiency property of an estimator),
3.  95%-Confidence Interval of ≈ **$(round.(quantile(estimation_results.β₁, [0.025, 0.975]), sigdigits = 3))** being estimated by OLS to be **$(round.(Array(DataFrame(coeftable(fitted_OLS))[2, 6:7]), sigdigits = 3))**.
""")

Key takeaways from sampling distribution of price OLS estimates

In [ ]:
println("The temperature impact coefficient (β₂) estimated by Ordianary Least Squares (OLS) at the value of ≈",
    round(DataFrame(coeftable(fitted_OLS))[3, 2],
        sigdigits = 3),
    " can be considered as a random variable derived from so called sampling distribution that we approximated (see an orange histogram) through Monte Carlo simulations with:\n",
    "(1) a mean of ≈",
    round(mean(estimation_results.β₂), sigdigits = 3),
    " equal to the true value of this parameter (unbiasedness property of an estimator),\n",
    "(2) a standard deviation (called standard error of this estimate) of ≈",
    round(std(estimation_results.β₂), sigdigits = 3),
    " being estimated by OLS to be ",
    round(DataFrame(coeftable(fitted_OLS))[3, 3],
        sigdigits = 3),
    " - the minimal possible standard error (efficiency property of an estimator),\n",
    "(3) 95%-Confidence Interval of ≈",
    round.(quantile(estimation_results.β₂, [0.025, 0.975]),
        sigdigits = 3),
    " being estimated by OLS to be ",
    round.(Array(DataFrame(coeftable(fitted_OLS))[3, 6:7]),
        sigdigits = 3),
    ".")

Key takeaways from sampling distribution of rainfall OLS estimates:

In [ ]:
println("The rainfall impact coefficient (β₃) estimated by Ordianary Least Squares (OLS) at the value of ≈",
    round(DataFrame(coeftable(fitted_OLS))[4, 2],
        sigdigits = 3),
    " can be considered as a random variable derived from so called sampling distribution that we approximated (see a green histogram) through Monte Carlo simulations with:\n",
    "(1) a mean of ≈",
    round(mean(estimation_results.β₃), sigdigits = 3),
    " equal to the true value of this parameter (unbiasedness property of an estimator),\n",
    "(2) a standard deviation (called standard error of this estimate) of ≈",
    round(std(estimation_results.β₃), sigdigits = 3),
    " being estimated by OLS to be ",
    round(DataFrame(coeftable(fitted_OLS))[4, 3],
        sigdigits = 3),
    " - the minimal possible standard error (efficiency property of an estimator),\n",
    "(3) 95%-Confidence Interval of ≈",
    round.(quantile(estimation_results.β₃, [0.025, 0.975]),
        sigdigits = 3),
    " being estimated by OLS to be ",
    round.(Array(DataFrame(coeftable(fitted_OLS))[4, 6:7]),
        sigdigits = 3),
    ".")

Sampling distribution of rainfall significance test p-values:

In [ ]:
histogram(estimation_results."β₃ p-value",
    xlabel = "p-value of rainfall test of significance",
    title = "Sampling distribution of
    rainfall (β₃) significance test p-values",
    legend = false)


Key takeaways from Sampling distribution of rainfall significance test p-values:

In [ ]:
print("Given the null hypothesis of no impact of rainfall being true, the sampling distribution of p-values will be uniform on [0, 1] - see a histogram above.")
println("As a result, using the significance level of α = 5% will result in rejecting the true null hypothesis (so called type I error or false positive) of no rainflall impact on demand as often as ",
    round(mean(estimation_results."β₃ p-value" .< 0.05), sigdigits = 3) * 100,
    "% of times (being equal to α = 5%)."
)

Key takeaways:
- It's unavoidable to make a type I error (false positive), so be aware of it, whenever you claim the impact of X, e.g. rainfall,  on Y, e.g. demand,
- You're controlling the probability of type I error (false positive) by setting up the significance level, e.g. α ∈ [1%, 5%, 10%],,
- However, there's no free lunch, but a trade-off, since the lower the probability of false positive, e.g. α ∈ [1%, 5%, 10%], the higher the probability of so called type II error (false negative), which is failing to reject a false null hypothesis. This type of error is higher the smaller data is, so you might want to choose higher α e.g. 5%, 10% for smaller data and smaller  α e.g. 1% or 5% for bigger data.

Key takeaways from Sampling distribution of R²

In [ ]:
println("The coefficient of determination (R²) estimated by Ordianary Least Squares (OLS) at the value of ≈",
    round(r2(fitted_OLS), sigdigits = 3),
    " can be considered as a random variable derived from so called sampling distribution that we approximated through Monte Carlo simulations with:\n",
    "(1) a mean of ≈",
    round(mean(estimation_results.R²), sigdigits = 3),
    "\n(2) a standard deviation (called standard error of this estimate) of ≈",
    round(std(estimation_results.R²), sigdigits = 3),
    "\n(3) 95%-Confidence Interval of ≈",
    round.(quantile(estimation_results.R², [0.025, 0.975]),
        sigdigits = 3),
    ".")

### g. Coefficient biasedness due to correlated omitted variable


Defining a simple but wrong specification omitting both temperature and rainfall:


In [ ]:
wrong_specification = @formula(demand ~ price)

Estimating a wrongly specified model:

In [ ]:
wrong_spec_model = lm(wrong_specification, dataset)

Initializing an empty array to be popoulated by price coefficients:

In [ ]:
biased_estimate_names =  ["β₁", "β₁ p-value"]
biased_estimates = DataFrame([column_name => Float64[] for column_name in biased_estimate_names])

Running Monte Carlo Simulations:

In [ ]:
for i in 1:N
  dataset = dgp() # data generation
  wrong_spec_model = lm(wrong_specification, dataset) # wrong model fitting
  push!(biased_estimates, [coef(wrong_spec_model)[2] DataFrame(coeftable(wrong_spec_model))[2, 5]])
end

Sampling distribution of price coefficient in a wrongly specified model:

In [ ]:
biased_estimates

In [ ]:
histogram(Matrix(biased_estimates),
    fillalpha=0.5,
    xlabel = "β₁ (price) estimate / p-value",
    ylabel = "Frequency",
    title = "Sampling distribution of rainfall (β₃)
    estimates & significance test p-values",
    labels = ["β₁ (price) estimates" "β₁ = 0 test p-values"])

Calculating a bias being a difference between the expected value of an estimate and a true value:

In [ ]:
bias =  mean(biased_estimates.β₁) - (-1)

Main takeaways from omitted variable problem:


In [ ]:
println("OLS estimate of price coefficient is biased, i.e. systematically being wrong with an expected value of ≈",
    round(mean(biased_estimates.β₁), sigdigits = 3),
    ".\nGiven a true price coefficient value of -1, the total bias is equal to (",
    round(mean(biased_estimates.β₁), sigdigits = 3),
    " - (-1)) = ", round(bias, sigdigits = 3))
println("This bias is due to the omitted variable, i.e. temperature impacting both price and demand, e.g. ommiting a rainfall doesn't result in a bias even though it is correlated with price but not impacting a demand.")
println("Since the temprerature is impacting both demand and price, when ommited its postive impact on demand is overtaken by the price, which results in a biased price impact estimation.") 


*Preparation of this workshop has been supported by the Polish National Agency for Academic Exchange under the Strategic Partnerships programme, grant number BPI/PST/2021/1/00069/U/00001.*

![SGH & NAWA](../logo.png)